## Imports

In [ ]:
%load_ext lab_black

import h5py
import os
import numpy as np
from typing import Dict, List, Optional, Tuple

from dataclasses import dataclass
from tqdm.auto import tqdm
from scipy.signal import savgol_filter
from scipy.interpolate import interp2d
from functools import lru_cache


import plotly.graph_objects as go
import plotly.colors as pc
import matplotlib.pyplot as plt


import sys

sys.path.append(r"C:\Users\atully\Code\GitHub\ARPES Code\arpes-code-python")
from arpes_functions import (
    fitting_functions,
    analysis_functions,
    plotting_functions,
    HDF5_loader,
    misc_functions,
    filter_functions,
    tr_functions,
    loading_functions,
    kw_data_loader,
    cnn,
    polygons,
)

colors = pc.qualitative.D3
angstrom = "\u212B"
Theta = "\u0398"
phi = "\u03C6"

In [ ]:
def average_timescans(files, ddir, new_filename):
    datas = []
    for i in range(0, len(files)):
        ad = ARPES_DATA[files[i]]
        datas.append(ad.data)
    data_avg = np.mean(datas, axis=0)
    print(data_avg.shape)

    new_data = data_avg

    new_fn = os.path.join(ddir, new_filename)

    with h5py.File(
        new_fn, "w"
    ) as f:  # Note: 'w' creates a new empty file (or overwrites), use 'r+' to modify an existing file
        f["data"] = new_data.T
        axes_names = [
            "angles",
            "energies",
        ]  # Change these to match your axes labels
        axes = [ad.theta, ad.energy]
        for axis, name in zip(axes, axes_names):
            f[name] = np.atleast_2d(axis).T
        entry_group = f.require_group("entry1")
        entry_group["ScanValues"] = np.atleast_2d(ad.phi_or_time).T
    return new_fn

# Load Data

In [ ]:
ddir = r"E:\atully\arpes_data\2023_May\May13\C60\FS"
files = []

## FS2 & FS0  # CT1, pos delay
# files = [f"FS2\FS_002_{i}.h5" for i in range(1, 15)]
# files.extend([f"FS0\FS_000_{i}.h5" for i in range(1, 19)])

## FS2  # CT1, pos delay
# files = [f"FS2\FS_002_{i}.h5" for i in range(1, 5)]  # for difference map

## FS3  # CT1, neg delay
# files = [f"FS3\FS_003_{i}.h5" for i in range(1, 5)]
# files.extend([f"" for i in range(1, )])

## FS4  # CT2, pos delay
# files = [f"FS4\FS_004_{i}.h5" for i in range(1, 4)]

## FS5  # CT1, pos delay --> centered on kx=0
files = [f"FS5\FS_005_{i}.h5" for i in range(1, 19)]

ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, kx, ky, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=kx, phi_or_time=ky, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

In [ ]:
# average_timescans(files, ddir, "FS0\FS0_Ali_avg.h5")
# average_timescans(files, ddir, "FS2\FS2_Ali_avg.h5")
# average_timescans(files, ddir, "FS0&FS2_Ali_avg.h5")
# average_timescans(files, ddir, "FS2_fordiffmap.h5")
# average_timescans(files, ddir, "FS3_Ali_avg.h5")
# average_timescans(files, ddir, "FS4_Ali_avg.h5")
average_timescans(files, ddir, "FS5_Ali_avg.h5")

In [ ]:
## Load averaged data ##

ddir = r"E:\atully\arpes_data\2023_May\May13\C60\FS"
files = []

# files = ["FS0\FS0_Ali_avg.h5"]  # CT1
# files = ["FS0\FS_000_1.h5"]  # CT1
# files = ["FS1\FS_001.h5"]  # CT1 -- 1 cycle with higher pump and probe powers
# files = ["FS2\FS2_Ali_avg.h5"]  # CT1, better pump effect and probe power

## CT1, pos delay, ad_right
files = ["FS0&FS2_Ali_avg.h5"]

## CT1, pos delay, centered at kx=0, ad_left
# files = ["FS5_Ali_avg.h5"]

## CT1, pos delay --> for diffmap
# files = ["FS2_fordiffmap.h5"]

## CT1, neg delay
# files = ["FS3_Ali_avg.h5"]

## CT2, pos delay
# files = ["FS4_Ali_avg.h5"]


## Feb Comparison ##
# ddir = r"E:\atully\arpes_data\2023_February\6eV\FS"
# files = []
# files = ["FS1_FS3_Ali_avg.h5"]  # 2.0 eV center energy
# files = ["FS_001_1.h5"]  # 2.0 eV center energy

# This works, but makes dataclass with theta and phi_or_time instead of kx and ky
ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, kx, ky, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=kx, phi_or_time=ky, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

ad = ARPES_DATA[files[0]]
# ad_may = ARPES_DATA[files[0]]
# ad_feb = ARPES_DATA[files[0]]

# ad_pos = ARPES_DATA[files[0]]
# ad_neg = ARPES_DATA[files[0]]

ad_right = ARPES_DATA[files[0]]
# ad_left = ARPES_DATA[files[0]]

In [ ]:
# ad = ARPES_DATA[files[0]]
# for k in ["energy", "theta", "phi_or_time"]:
#     print(f"{k}.shape = {getattr(ad, k).shape}")
# print(f"Data.shape = {ad.data.shape}")

# Analysis

In [ ]:
slice_dim = "y"
# slice_val = 0
int_range = 0.05
xlim = (-18, 16)
ylim = None
x_bin = 2
y_bin = 2

# Load Data

# for i, v in enumerate([3.0, 2.9, 2.8, 2.7, 2.6, 2.5, 2.4]):  # CT2
for i, v in enumerate([2.4, 2.3, 2.2, 2.1, 2.0, 1.9]):  # CT1
    x, y, d = tr_functions.slice_datacube(
        ad_dataclass=ad_left,
        slice_dim=slice_dim,
        slice_val=v,
        int_range=int_range,
        xlim=xlim,
        ylim=ylim,
        x_bin=x_bin,
        y_bin=y_bin,
        norm_data=True,
        plot_data=False,
    )

    ## Plot Data MPL
    # fig, ax = plotting_functions.plot_2D_mpl(
    #     x=x,
    #     y=y,
    #     data=d,
    #     xlabel="theta",
    #     ylabel="phi",
    #     title=f"Constant Energy Cut: {v} eV",
    #     # cmap="gray",
    #     cmap="Blues",
    # )
    # ratio = 1  # set aspect ratio
    # x_left, x_right = ax.get_xlim()
    # y_low, y_high = ax.get_ylim()
    # ax.set_aspect(abs((x_right - x_left) / (y_low - y_high)) * ratio)

    ## Plot Data Plotly
    fig = tr_functions.thesis_fig(
        title=f"E<sub>K</sub> = {v}",
        xaxis_title=f"{Theta}",
        yaxis_title=f"{phi}",
        equiv_axes=False,
        height=500,
        width=500,
    )

    fig.add_trace(
        go.Heatmap(x=x, y=y, z=analysis_functions.norm_data(d), coloraxis="coloraxis")
    )

    if xlim is not None:
        fig.update_xaxes(range=[xlim[0], xlim[1]], constrain="domain")

    fig.update_coloraxes(cmin=0, cmax=1)

    fig.show()

# Stitch Datasets

In [ ]:
slice_dim = "y"
# slice_val = 0
int_range = 0.05
xlim = (-18, 16)
# xlim = None
ylim = None
x_bin = 2
y_bin = 2

for i, v in enumerate([2.4, 2.3, 2.2, 2.1, 2.0, 1.9]):  # CT1
    x_left, y_left, d_left = tr_functions.slice_datacube(
        ad_dataclass=ad_left,
        slice_dim=slice_dim,
        slice_val=v,
        int_range=int_range,
        xlim=xlim,
        ylim=ylim,
        x_bin=x_bin,
        y_bin=y_bin,
        norm_data=True,
        plot_data=False,
    )
    x_left = x_left - (14.2 - 4.5)

    x_right, y_right, d_right = tr_functions.slice_datacube(
        ad_dataclass=ad_right,
        slice_dim=slice_dim,
        slice_val=v,
        int_range=int_range,
        xlim=xlim,
        ylim=ylim,
        x_bin=x_bin,
        y_bin=y_bin,
        norm_data=True,
        plot_data=False,
    )

    # xs, ys, ds = tr_functions.stitch_and_avg(
    #     x_left,
    #     y_left,
    #     d_left,
    #     x_right,
    #     y_right,
    #     d_right,
    #     no_avg=False,
    # )

    # xs, ys, ds = tr_functions.stitch_2_datasets(
    # d_left, x_left, y_left, d_right, x_right, y_right, stitch_dim="x"
    # )

    ## Plot Data

    fig = tr_functions.thesis_fig(
        title=f"E<sub>K</sub> = {v}",
        xaxis_title=f"{Theta}",
        yaxis_title=f"{phi}",
        equiv_axes=False,
        height=500,
        width=500,
    )
    
    fig.add_trace(
        go.Heatmap(
            x=x_left,
            y=y_left,
            z=analysis_functions.norm_data(d_left),
            coloraxis="coloraxis",
        )
    )

    fig.add_trace(
        go.Heatmap(
            x=x_right,
            y=y_right,
            z=analysis_functions.norm_data(d_right),
            coloraxis="coloraxis",
        )
    )

    # fig.add_trace(
    #     go.Heatmap(
    #         x=xs, y=ys, z=analysis_functions.norm_data(ds), coloraxis="coloraxis"
    #     )
    # )

    # if xlim is not None:
    #     fig.update_xaxes(range=[xlim[0], xlim[1]], constrain="domain")

    fig.update_coloraxes(cmin=0, cmax=1)

    fig.show()

# Angular Dispersion: Ek vs theta

In [ ]:
## Set Initial Parameters ##

slice_dim = "z"

slice_val = -4
int_range = 1

xlim = (-18, 16)
# xlim = None

## Looking for cone or dispersion
ylim = (1.93, 2.32)  # Feb
# ylim = (1.9, 2.44)  # May
# ylim = None

x_bin = 1
y_bin = 1

In [ ]:
x, y, d = tr_functions.slice_datacube(
    ad_dataclass=ad,
    slice_dim=slice_dim,
    slice_val=slice_val,
    int_range=int_range,
    xlim=xlim,
    ylim=ylim,
    x_bin=x_bin,
    y_bin=y_bin,
    norm_data=True,
    plot_data=False,
)

In [ ]:
## Plot Data ##

fig = tr_functions.thesis_fig(
    title=f"{phi} = {slice_val} degrees",
    xaxis_title=f"{Theta}",
    yaxis_title="E<sub>K</sub>",
    equiv_axes=False,
)

fig.add_trace(
    go.Heatmap(x=x, y=y, z=analysis_functions.norm_data(d), coloraxis="coloraxis")
)

if xlim is not None:
    fig.update_xaxes(range=[xlim[0], xlim[1]], constrain="domain")

fig.update_coloraxes(cmin=0, cmax=0.2)

fig.show()

# EDCs

In [ ]:
## Integrate over desired angular range ##

slice_dim = "x"
slice_val = 0
int_range = 50  # if this value is more that the integration range, my get_2D_slice function will just integrate over the max range.

xlim = None
ylim = None

x_bin = 1
y_bin = 1

In [ ]:
fig = tr_functions.thesis_fig(
    title=f"EDC",
    xaxis_title=f"E<sub>K</sub>",
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
)

names = []
names = ["feb", "may"]
for i, ad in enumerate([ad_feb, ad_may]):
    x_2d, y_2d, d_2d = tr_functions.slice_datacube(
        ad_dataclass=ad,
        slice_dim=slice_dim,
        slice_val=slice_val,
        int_range=int_range,
        xlim=xlim,
        ylim=ylim,
        # ylim=(
        #     ad.energy[57],
        #     ad.energy[1007],
        # ),  # get rid of zero padding on datasets
        x_bin=x_bin,
        y_bin=y_bin,
        norm_data=False,
        plot_data=False,
    )

    xlim = None

    y_1d, col = tr_functions.get_1d_x_slice(
        x=x_2d,
        y=y_2d,
        data=d_2d,
        ylims=ylim,
        x_range=xlim,
    )

    # if normalize:
    #     col = analysis_functions.norm_data(col)

    # if logplot:
    #     col = np.log(col)

    # Plot Data
    color = colors[i % len(colors)]
    fig.add_trace(go.Scatter(x=y_1d, y=col, name=names[i], line=dict(color=color)))

    # Plot Data
    # fig.add_trace(
    #     go.Scatter(
    #         x=y_1d,
    #         y=col,
    #         name=f"data",
    #         line=dict(color=colors[0]),
    #     )
    # )

fig.show()

In [ ]:
## Plot EDC for Fit ##

xlim = None


fig = tr_functions.thesis_fig(
    title=f"EDC",
    xaxis_title=f"{Theta}",
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
)

y_1d, col = tr_functions.get_1d_x_slice(
    x=x_2d,
    y=y_2d,
    data=d_2d,
    ylims=ylim,
    x_range=xlim,
)

# if normalize:
#     col = analysis_functions.norm_data(col)

# if logplot:
#     col = np.log(col)

# Plot Data
fig.add_trace(
    go.Scatter(
        x=y_1d,
        y=col,
        name=f"data",
        line=dict(color=colors[0]),
    )
)

fig.show()

In [ ]:
# ## Load K corrected, negative delay data ##

# ddir = r"E:\atully\arpes_data\2023_February\6eV\FS"
# files = ["FS_4and10_gkw.h5"]

# # This works, but makes dataclass with theta and phi_or_time instead of kx and ky
# ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
# ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
# for file in tqdm(files):
#     data, kx, ky, energy = loading_functions.load_hdf5(ddir, file)
#     ARPES_DATA[file] = tr_functions.ArpesData(
#         data=data, theta=kx, phi_or_time=ky, energy=energy
#     )
#     ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

# Diff Maps

In [ ]:
# ad_pos = ARPES_DATA[files[0]]
# for k in ["energy", "theta", "phi_or_time"]:
#     print(f"{k}.shape = {getattr(ad_pos, k).shape}")
# print(f"Data.shape = {ad_pos.data.shape}")

In [ ]:
# ad_neg = ARPES_DATA[files[0]]
# for k in ["energy", "theta", "phi_or_time"]:
#     print(f"{k}.shape = {getattr(ad_neg, k).shape}")
# print(f"Data.shape = {ad_neg.data.shape}")

In [ ]:
## Check delays of relevent scans ##

time_zero = 37.8

print(
    f"positive delay (FS0&FS2) = {np.round(tr_functions.mm_to_ps(37.86, time_zero), 3)} ps"
)

# print(
#     f"negative delay (FS4&10) = {np.round(tr_functions.mm_to_ps(37.81, time_zero), 3)} ps"
# )

In [ ]:
# ## HOMO is at 2.05 eV below EF, based on fits from this data averaged with fits from tr-ARPES results ##

# EF_400 = 1.91  # in kinetic energy, slit 400

# homo = -2.05

# homo_400 = homo + EF_400

In [ ]:
# ad_pos.energy = ad_pos.energy - homo_400

In [ ]:
# ad_neg.energy = ad_neg.energy - homo_400

In [ ]:
## Slicevals ##
# slice_val = np.round(2.15 - homo_400, 3)
# slice_val = np.round(2.05 - homo_400, 3)
# slice_val = np.round(1.95 - homo_400, 3)

In [ ]:
# title = f"CT<sub>1</sub> (E - E<sub>HOMO</sub> = {slice_val})"
title = f"CT<sub>1</sub> (E<sub>K</sub> = {slice_val})"
# yaxis_title = f"k<sub>y</sub> [{angstrom}<sup>-1</sup>]"
yaxis_title = f"{phi}"
# xaxis_title = f"k<sub>x</sub> [{angstrom}<sup>-1</sup>]"
xaxis_title = f"{Theta}"

In [ ]:
slice_dim = "y"
# slice_val = 0
int_range = 0.05
xlim = (-20, 16)
# xlim = None
ylim = None
x_bin = 1
y_bin = 1

# Load Data

# for i, v in enumerate([3.0, 2.9, 2.8, 2.7, 2.6, 2.5, 2.4]):  # CT2
for i, v in enumerate([2.4, 2.3, 2.2, 2.1, 2.0, 1.9]):  # CT1
    x_pos, y_pos, d_pos = tr_functions.slice_datacube(
        ad_dataclass=ad_pos,
        slice_dim=slice_dim,
        slice_val=v,
        int_range=int_range,
        xlim=xlim,
        ylim=ylim,
        x_bin=x_bin,
        y_bin=y_bin,
        norm_data=False,
        plot_data=False,
    )

    x_neg, y_neg, d_neg = tr_functions.slice_datacube(
        ad_dataclass=ad_neg,
        slice_dim=slice_dim,
        slice_val=v,
        int_range=int_range,
        xlim=xlim,
        ylim=ylim,
        x_bin=x_bin,
        y_bin=y_bin,
        norm_data=False,
        plot_data=False,
    )

    d_diff = d_pos - d_neg
    x_diff, y_diff = x_pos, y_pos

    ## Plot Data Plotly
    fig = tr_functions.thesis_fig(
        title=f"E<sub>K</sub> = {v}",
        xaxis_title=f"{Theta}",
        yaxis_title=f"{phi}",
        equiv_axes=False,
        height=500,
        width=500,
    )

    fig.add_trace(go.Heatmap(x=x_diff, y=y_diff, z=d_diff, coloraxis="coloraxis"))

    if xlim is not None:
        fig.update_xaxes(range=[xlim[0], xlim[1]], constrain="domain")

    fig.update_coloraxes(colorscale="RdBu", cmid=0, showscale=True)
    # fig.update_coloraxes(cmin=-0.2, cmax=0.2)

    fig.show()

## Positive Delay

In [ ]:
## Get Data ##

x_pos, y_pos, d_pos = tr_functions.slice_datacube(
    ad_dataclass=ad_pos,
    slice_dim=slice_dim,
    slice_val=slice_val,
    int_range=int_range,
    xlim=xlim,
    ylim=ylim,
    x_bin=x_bin,
    y_bin=y_bin,
    norm_data=True,
    plot_data=False,
)

In [ ]:
## Plot Data ##

fig = tr_functions.thesis_fig(
    title=f"Positive Delay: {slice_val} eV",
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=True,
    height=500,
    width=500,
)

fig.add_trace(
    go.Heatmap(
        x=x_pos, y=y_pos, z=analysis_functions.norm_data(d_pos), coloraxis="coloraxis"
    )
)

# hexagon = polygons.gen_polygon(6, 0.42, rotation=30)
# fig = polygons.plot_polygon(
#     hexagon, color="firebrick", fig=fig, show=False, dash=True, dash_width=3
# )

if xlim is not None:
    fig.update_xaxes(range=[xlim[0], xlim[1]], constrain="domain")

fig.update_coloraxes(cmin=0, cmax=1)

fig.show()